## Joining data with other attributes based ID 

In [2]:
import arcpy
import os
import pandas as pd
import xlsxwriter
from arcpy import metadata as md


##  Inputs parameters

In [34]:
excel_file = r"\\WM20ocqu46ph01\WF_Projects\NL_HKW\1_INPUT\VESSELS\20240226_InstallationLogistics_LPEA_Javier_EditedDocuments\1.xlsx"
excel_sheet = "Sheet1"

# Set the workspace environment
arcpy.env.workspace = r"\\WM20ocqu46ph01\WF_Projects\NL_HKW\2_FINAL\VESSELS\VESSELS.gdb"  # Update the path to your geodatabase

# Input feature class
feature_class = "HKW_VII_LPEA_Vessel_Voltaire_L09_236D_56T_UTM31N_v0"  # Update with the name of your feature class


## 1) Excel file - reading and editing if needed 

In [35]:
# Read Excel file into a pandas DataFrame
df = pd.read_excel(excel_file)
df = df.iloc[:,0:-3]
# df = df.iloc[:,[0,-3]]
df.head(1)

c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():


,WTG ID,Link to LPEA Annex A
0,HWB 01,https://rwe.sharepoint.com/teams/HKW/Shared%20...


### 2) Excel file - saved modified version in gdb

In [36]:

output_table = os.path.join(os.path.dirname(excel_file), "modified_table.xls")
df.to_excel(output_table, index=False)

gdb_table_path = os.path.join(arcpy.env.workspace , "excel_file") 
if arcpy.Exists(gdb_table_path):
    # Delete the previous table in the geodatabase
    arcpy.Delete_management(gdb_table_path)
    print("Deleting excel_file")
   
# Convert modified Excel file to a table
arcpy.ExcelToTable_conversion(output_table, "excel_file")
print("New Excel file was created in geodatabase - excel_file")

# Get a list of field objects
fields_feature_class = arcpy.ListFields("excel_file")

# Iterate over the fields and print their names
for field in fields_feature_class:
    print(field.name)

C:\Users\ui150920\AppData\Local\Temp\1\ipykernel_14584\964271733.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel(output_table, index=False)


Deleting excel_file
New Excel file was created in geodatabase - excel_file
OBJECTID
WTG_ID
Link_to_LPEA_Annex_A


### 3) Read feature class columns to find ID

In [37]:
# Get a list of field objects
gdb_feature = os.path.join(arcpy.env.workspace , feature_class) 
fields_feature_class = arcpy.ListFields(gdb_feature)

# Iterate over the fields and print their names
for field in fields_feature_class:
    print(field.name)

OBJECTID
Shape
Coord_adapted
LAYOUT_ID
NAME_OLD
WDPTH_LAT
NAME
INTERFACE_INNOVATION
X_ETRS_UTM31
Y_ETRS_UTM31
X_ETRS
Y_ETRS
Aviation_lights_medium
Obstruction_lights_on_Tower
Visibility_meter_WTG
Marine_light_SPS_MAP
Marine_light_IPS_MAP
Fog_horn_MAP
Visibility_meter_MAP
NEAR_FID
NEAR_DIST
NEAR_X
NEAR_Y
NEAR_ANGLE
WTG_ID
Water_Depth__mLAT_
Expected_spudcan_tip_penetration_without_sand_plug__mbsf__VOLTAI
HAZARD_Insufficient_leg_length
HAZARD_Insufficient_soil_data
Hang_up_RISK
Punching_RISK
Spudcan_extraction_RISK
Hard_seabed_RISK
HAZARD_Interaction_with_other_strcutures


### 4) Join the DataFrame to the feature class

In [38]:
arcpy.JoinField_management(gdb_feature, "NAME",gdb_table_path, "WTG_ID")
print("DataFrame joined to feature class successfully.")

# Let check JOIN feature class
data = []
field_names = [field.name for field in arcpy.ListFields(feature_class)]
with arcpy.da.SearchCursor(feature_class, field_names) as cursor:
    for row in cursor:
        data.append(row)
        
df = pd.DataFrame(data, columns=field_names)
df.head(5)

DataFrame joined to feature class successfully.


,OBJECTID,Shape,Coord_adapted,LAYOUT_ID,NAME_OLD,WDPTH_LAT,NAME,INTERFACE_INNOVATION,X_ETRS_UTM31,Y_ETRS_UTM31,...,Expected_spudcan_tip_penetration_without_sand_plug__mbsf__VOLTAI,HAZARD_Insufficient_leg_length,HAZARD_Insufficient_soil_data,Hang_up_RISK,Punching_RISK,Spudcan_extraction_RISK,Hard_seabed_RISK,HAZARD_Interaction_with_other_strcutures,WTG_ID_1,Link_to_LPEA_Annex_A
0,1,"(542448.4000000004, 5832170.0)",X+0.4m,HKW_VII_WTG_L04_A2_60xSG15MW_UTM31N_v0,1,-29.972,HWB 26,None,542448.4,5832170.0,...,1.4,Low,-,-,High,-,-,-,HWB 26,https://rwe.sharepoint.com/teams/HKW/Shared%20...
1,2,"(547005.0, 5828296.0)",None,HKW_VII_WTG_L04_A2_60xSG15MW_UTM31N_v0,2,-28.807,HWB 56,None,547005.0,5828296.0,...,0.8,Low,-,-,Low,-,-,-,HWB 56,https://rwe.sharepoint.com/teams/HKW/Shared%20...
2,3,"(547139.0, 5837774.0)",None,HKW_VII_WTG_L04_A2_60xSG15MW_UTM31N_v0,3,-28.006,HWB 03,None,547139.0,5837774.0,...,0.8,Low,-,-,Low,-,-,-,HWB 03,https://rwe.sharepoint.com/teams/HKW/Shared%20...
3,4,"(540678.0, 5830049.0)",None,HKW_VII_WTG_L04_A2_60xSG15MW_UTM31N_v0,4,-32.495,HWB 39,None,540678.0,5830049.0,...,3.6,Low,-,-,Medium,High,-,-,HWB 39,https://rwe.sharepoint.com/teams/HKW/Shared%20...
4,5,"(552715.0, 5830475.5)",Y+0.5m,HKW_VII_WTG_L04_A2_60xSG15MW_UTM31N_v0,5,-31.018,HWB 45,None,552715.0,5830475.5,...,0.9,Low,-,-,Low,-,-,-,HWB 45,https://rwe.sharepoint.com/teams/HKW/Shared%20...


### 5) Delete Fields in a feature class [Optional]

In [154]:
# Get list of field names
field_names = [field.name for field in arcpy.ListFields(feature_class)]

# Determine the fields to delete
# fields_to_delete = field_names[-46:]
# fields_to_delete

### 5) Deleting file created in gdb!!!


In [39]:
arcpy.Delete_management(gdb_table_path)

<Result 'true'>